In [ ]:
#LIMITATIONS:
#    50 results per query
#    Unable to filter by ratings (no input parameter available)

In [21]:
# Import required dependencies
import json
import time
import csv
import requests
import pandas as pd
from pprint import pprint
from config import api_key, client_id

In [22]:
#Prepare empty lists to store all data
restaurant_name = []
restaurant_id = []
restaurant_rating = []
restaurant_reviewcount = []
restaurant_city = []
restaurant_state = []
restaurant_zipcode = []
restaurant_price=[]
restaurant_type = []
restaurant_category = []

#Necessary to pass header in each API request.  Do not change this.
headers = {'Authorization': f"Bearer {api_key}"}
base_url = 'https://api.yelp.com/v3/businesses/search?'
base_filters = '&categories=Restaurants&limit=50&is_closed=0'

# Construct the URL to request Yelp API data

                #San Francisco, CA
url_location = ['94123', '94115', '94114', '94105', '94133', 
                #New York, NY
               '10025', '10002', '10029', '10023', '10009',
                #Nebraska (Abie, Craig, Memphis, South Bend, Washington)
               '68001', '68019', '68042', '68058', '68068',
                #Iowa (Adair, Baxter, Derby, Liscomb, Rhodes)
               '50002', '50028', '50068', '50148', '50234']

url_price = [1, 2, 3, 4]

In [17]:
#Yelp provides us with a list of 50 random results per query

#loop through list of rural and suburb locations
for location in url_location:
    
    #loop through list of prices $-$$$$
    for price in url_price:
        
        #prepare query URL for each price range and each zip code region
        query_url = f'{base_url} &location= {location} &price= {price} {base_filters}'
        
        #extract data into a response variable
        response = requests.get(query_url, headers=headers).json()
        
        restaurant_list = response['businesses']
        
        #loop through responses and append data to empty lists
        for restaurant in restaurant_list:
            restaurant_name.append(restaurant["name"])
            restaurant_id.append(restaurant["id"])
            restaurant_rating.append(restaurant["rating"])
            restaurant_reviewcount.append(restaurant["review_count"])
            restaurant_price.append(len(restaurant["price"]))
            restaurant_city.append(restaurant["location"]["city"])
            restaurant_state.append(restaurant["location"]["state"])
            restaurant_zipcode.append(restaurant["location"]["zip_code"])
            restaurant_type.append(restaurant["categories"][0]["title"])

        #prepare dataframe from dictionary
        restaurant_df = pd.DataFrame({'Restaurant':restaurant_name, 
                           'ID':restaurant_id,
                           'Rating':restaurant_rating, 
                           'Reviews': restaurant_reviewcount, 
                           'Price':restaurant_price, 
                           'City': restaurant_city,
                           'State': restaurant_state,
                           'ZipCode':restaurant_zipcode,
                           #need to be able to derive multiple categories
                           'Category':restaurant_type})

        restaurant_df.head()

In [29]:
#cleanse data by deleting any NULL records
restaurant_df.dropna(how='any')
restaurant_df.count()

Restaurant    2402
ID            2402
Rating        2402
Reviews       2402
Price         2402
City          2402
State         2402
ZipCode       2402
Category      2402
dtype: int64

In [26]:
#investigate data by outputting results to CSV file
restaurant_df.to_csv("Restaurant_Info.csv", index=False, encoding='utf-8')

In [30]:
restaurant_df['ZipCode'].value_counts()

94115    124
10002    116
94133    113
94123     99
10023     97
94109     87
10029     83
10024     75
94110     69
10003     68
94102     68
10009     67
10025     63
94103     59
10012     51
94108     49
94114     42
50208     36
50309     35
68508     34
10019     33
50158     29
94111     29
10128     28
68028     28
10013     27
50266     26
68130     24
68102     24
68118     23
        ... 
50047      1
50120      1
10014      1
68502      1
50220      1
51566      1
50210      1
52577      1
10038      1
68050      1
10292      1
68066      1
68648      1
50313      1
51577      1
68016      1
50164      1
50621      1
50135      1
94129      1
50166      1
10069      1
07047      1
68029      1
64442      1
50613      1
68403      1
50067      1
50033      1
50274      1
Name: ZipCode, Length: 190, dtype: int64